In [1]:
import numpy
import pandas
import os

row_list = []
for subdir in ['bagus','jelek']:
    for folder, subfolders, filenames in os.walk('data/'+subdir):
        for file in filenames:  
            d = {'class':subdir}  
            with open('data/'+subdir+'/'+file) as f:
                if f.read():      
                    f.seek(0)
                    d['text'] = f.read()  
            row_list.append(d)
        break


In [2]:
dataframe = pandas.DataFrame(row_list)
print(dataframe)

     class                                               text
0    bagus  do film critics have morals ? \nare there any ...
1    bagus  this sunday afternoon i had the priviledge of ...
2    bagus  note : some may consider portions of the follo...
3    bagus  after a stylistic detour with mrs . \nparker a...
4    bagus  i was pleasantly surprised by this film . \nwi...
..     ...                                                ...
995  jelek   " showgirls " is the first big-budget , big-s...
996  jelek  it is movies like these that make a jaded movi...
997  jelek  it would be hard to choose the best american p...
998  jelek  studio 54 attracted so many weird and bizarre ...
999  jelek  sean connery stars as a harvard law professor ...

[1000 rows x 2 columns]


In [3]:
dataframe.head()

,class,text
0,bagus,do film critics have morals ? \nare there any ...
1,bagus,this sunday afternoon i had the priviledge of ...
2,bagus,note : some may consider portions of the follo...
3,bagus,after a stylistic detour with mrs . \nparker a...
4,bagus,i was pleasantly surprised by this film . \nwi...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(dataframe['text'])


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [5]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = dataframe['class']
classifier.fit(counts, targets)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [6]:
examples = ["the law of crowd pleasing romantic movies states that the two leads must end up togetherby film's end .if you're not familiar with this law , then maybe you've seen the trailer for this film which shows that the two leads are together by film's end . now if you're a regular reader of mine , you've heard me say this countless times : you know how drive me crazy is going to end , but is the journey to get to that ending worth it ? no , it definitely is not . melissa joan hart ( from abc's  sabrina , the teenage witch  ) likes a hunky stud on the basketball team . adrien grenier is her grungy neighbor who's just broken up with his activist girlfriend . apparently he wants to make his ex-girlfriend jealous enough to take him back , and she wants someone to take her to the big year end dance .", "this three hour movie opens up with a view of singer/guitar player/musician/composer frank zappa rehearsing with his fellow band members . all the rest displays a compilation of footage , mostly from the concert at the palladium in new york city , halloween 1979 . other footage shows backstage foolishness , and amazing clay animation by bruce bickford . the performance of \" titties and beer \" played in this movie is very entertaining , with drummer terry bozzio supplying the voice of the devil . frank's guitar solos outdo any van halen or hendrix i've ever heard . bruce bickford's outlandish clay animation is that beyond belief with zooms , morphings , etc . and actually , it doesn't even look like clay , it looks like meat ."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
print(predictions)


['jelek' 'bagus']


In [7]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(dataframe['text'], dataframe['class'])
pipeline.predict(examples)


array(['jelek', 'bagus'], dtype='<U5')

In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score

k_fold = KFold(n_splits=6, random_state=None, shuffle=True)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold.split(dataframe):
    train_text = dataframe.iloc[train_indices]['text']
    train_y = dataframe.iloc[train_indices]['class']

    test_text = dataframe.iloc[test_indices]['text']
    test_y = dataframe.iloc[test_indices]['class']

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label="jelek")
    scores.append(score)

print('Total review classified:', len(dataframe))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=np.int)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/model_selection/_split.py:102: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Depreca

Total review classified: 1000
Score: 0.8145906522402505
Confusion matrix:
[[393 107]
 [ 82 418]]


In [9]:
tes = ["the law of crowd pleasing romantic movies states that the two leads must end up togetherby film's end .if you're not familiar with this law , then maybe you've seen the trailer for this film which shows that the two leads are together by film's end . now if you're a regular reader of mine , you've heard me say this countless times : you know how drive me crazy is going to end , but is the journey to get to that ending worth it ? no , it definitely is not . melissa joan hart ( from abc's  sabrina , the teenage witch  ) likes a hunky stud on the basketball team . adrien grenier is her grungy neighbor who's just broken up with his activist girlfriend . apparently he wants to make his ex-girlfriend jealous enough to take him back , and she wants someone to take her to the big year end dance ."]
tes_counts = count_vectorizer.transform(tes)
predictions = classifier.predict(tes_counts)
print(predictions)

['jelek']
